# PyCity Schools Analysis


In [165]:
import pandas as pd

In [166]:
file_school = "Resources/schools_complete.csv"
file_student = "Resources/students_complete.csv"

In [167]:
school_data = pd.read_csv(file_school)
student_data = pd.read_csv(file_student)

In [168]:
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [169]:
# total number of unique schools
school_count = school_data_complete["school_name"].unique().size

In [170]:
# the total number of students
student_count = school_data_complete["student_name"].count()

In [172]:
# the total budget
total_budget_unique = school_data_complete["budget"].unique()

total_budget = total_budget_unique.sum()

In [173]:
# the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()

In [174]:
# the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()

In [175]:
# the percentage of students who passed math (math scores greather than or equal to 70)

passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

In [176]:
# the percentage of students who passed reading   
passing_reading_count = school_data_complete[(school_data_complete["reading_score"]>=70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count)*100

In [177]:
# the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

In [178]:
# snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools": [school_count], "Total Students": [student_count],
                                 "Total Budget" : [total_budget],
                                "Average Math Score": [average_math_score], 
                                 "Average Reading Score" : [average_reading_score],
                                "% Passing Math": [passing_math_percentage], 
                                 "% Passing Reading": [passing_reading_percentage],
                                "% Overall Passing" : [overall_passing_rate] })


district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)


district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [179]:
# selects the school type
school_types = school_data.set_index(["school_name"])["type"]

In [180]:
# the total student count
per_school_counts = school_data_complete.groupby("school_name").count()["student_name"]

In [181]:
# the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [183]:
# the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]

per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

In [184]:
# the number of schools with math scores of 70 or higher

passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]

school_passing_math = passing_math.groupby(["school_name"]).count()["math_score"]

school_passing_math = school_passing_math.value_counts().sum()

In [185]:
# the number of schools with reading scores of 70 or higher
passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

school_passing_reading = passing_reading.groupby(["school_name"]).count()["reading_score"]

school_passing_reading = school_passing_reading.value_counts().sum()

In [186]:
# the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

school_passing_math_and_reading = passing_math_and_reading.groupby(["school_name"]).count()[["math_score","reading_score"]]
    
school_passing_math_and_reading = school_passing_math_and_reading.value_counts().sum()


In [ ]:
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)] 
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)] 
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)
                                               & (school_data_complete["math_score"] >= 70)]

In [187]:
# the overall passing rate
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [188]:
# data frame of all calculations per school
per_school_summary = pd.DataFrame({"Type" : school_types,
                                  "Total Students": per_school_counts,
                                   "Total School Budget" : per_school_budget,
                                    "Per Student Budget": per_school_capita,
                                  "Average Math Score": per_school_math,
                                  "Average Reading Score": per_school_reading,
                                  "% Passing Math": per_school_passing_math,
                                  "% Passing Reading": per_school_passing_reading,
                                  "% Overall Passing": overall_passing_rate})


per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary.index.names = [" "]

per_school_summary

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [189]:
# sorts the schools by `% Overall Passing` in descending order and displays the top 5 rows
high_performing_schools = per_school_summary.sort_values("% Overall Passing", ascending = False)
high_performing_schools.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [190]:
# sorts the schools by `% Overall Passing` in ascending order and displays the top 5 rows
low_performing_schools = per_school_summary.sort_values("% Overall Passing", ascending = True)
low_performing_schools.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores By Grade

In [230]:
# separates the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]


In [231]:
# groups by "school_name" and takes the mean of each

ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()


In [236]:
# selects only the `math_score`

ninth_grader_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]


In [237]:
# combines each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
                                    "9th": ninth_grader_math_scores,
                                    "10th": tenth_grader_math_scores,
                                    "11th" : eleventh_grader_math_scores,
                                    "12th" : twelfth_grader_math_scores})


math_scores_by_grade.index.name = None



math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores By Grade

In [238]:
# separates the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [239]:
# groups by "school_name" and takes the mean of each
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

In [240]:
# selects only the `reading_score`

ninth_grader_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

In [241]:
# combines each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
                                    "9th": ninth_grader_reading_scores,
                                    "10th": tenth_grader_reading_scores,
                                    "11th" : eleventh_grader_reading_scores,
                                    "12th" : twelfth_grader_reading_scores})


reading_scores_by_grade.index.name = None


reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores By School Spending

In [201]:
# establishs the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [202]:
# a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [203]:
school_spending_df.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [206]:
# used `pd.cut` to categorize spending based on the bins
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels = labels, right = False)

school_spending_df

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680


## Scores By School Size

In [208]:
# establishs the bins
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [212]:
# used `pd.cut` to categorize size based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels = labels, right = False)

per_school_summary

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)


## Scores By School Type

In [199]:
# groups the per_school_summary DataFrame by "School Type" and averages the results.

type_math_scores = school_data_complete.groupby(["type"]).mean()["math_score"]
type_reading_scores = school_data_complete.groupby(["type"]).mean()["reading_score"]
type_passing_math = per_school_summary.groupby(["Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary.groupby(["Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["Type"]).mean()["% Overall Passing"]

In [200]:
types = school_data_complete.set_index("type")

type_summary = pd.DataFrame({"Average Math Score": type_math_scores,
                            "Average Reading Score" : type_reading_scores,
                            "% Passing Math" : type_passing_math,
                            "% Passing Reading" : type_passing_reading,
                            "% Overall Passing": type_overall_passing})

type_summary.index.names = ["School Type"]

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.620830,96.586489,90.432244
District,76.987026,80.962485,66.548453,80.799062,53.672208
